In [132]:
# !pip install -qU langsmith langchain-core langchain-community langchain-openai langchain-qdrant


In [133]:
# !pip install -qU pymupdf ragas


In [134]:
# pip install -U langchain langchain-openai

In [135]:
import chainlit as cl
import os
import tiktoken
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema import Document
import pandas as pd

from dotenv import load_dotenv
import getpass
import os

In [136]:


from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")


LangChain API Key: ········


In [137]:
os.environ["LANGCHAIN_PROJECT"] = "AIE4 - Midterm - {uuid4().hex[0:8]}"



In [138]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")


OpenAI API Key: ········


## Setup RAG System

In [139]:
def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o-mini").encode(text)
    return len(tokens)

In [140]:
## Chunking Method Token and Splitting by Paragraph

def load_and_split_pdfs_by_paragraphs(directory, chunk_size=750, chunk_overlap=25):
    documents = []
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=tiktoken_len,
        separators=["\n\n", "\n"]
    )
    
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            file_path = os.path.join(directory, filename)
            
            loader = PyMuPDFLoader(file_path)
            pdf_documents = loader.load()
            
            for page_num, page in enumerate(pdf_documents):
                splits = text_splitter.split_text(page.page_content)
                for i, split in enumerate(splits):
                    documents.append(Document(
                        page_content=split,
                        metadata={
                            "filename": filename,
                            "page_number": page_num + 1,
                            "chunk_number": i + 1
                        }
                    ))
    
    return documents

In [141]:
# Set the directory containing your PDF files
current_directory = os.getcwd()  # Update this path
current_directory

'/Users/danielgutierrez/Midterm'

In [142]:
# def load_and_split_pdfs_by_chunks(directory, chunk_size=500, chunk_overlap=25):
#     documents = []
    
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=chunk_overlap,
#         length_function=len
#     )
    
#     for filename in os.listdir(directory):
#         if filename.endswith('.pdf'):
#             file_path = os.path.join(directory, filename)
            
#             loader = PyMuPDFLoader(file_path)
#             pdf_documents = loader.load()
            
#             for page_num, page in enumerate(pdf_documents):
#                 splits = text_splitter.split_text(page.page_content)
#                 for i, split in enumerate(splits):
#                     documents.append(Document(
#                         page_content=split,
#                         metadata={
#                             "filename": filename,
#                             "page_number": page_num + 1,
#                             "chunk_number": i + 1
#                         }
#                     ))
    
#     return documents

In [143]:
# RAG prompt template
RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

You are a helpful assistant. Use the available context to answer the question. If you can't answer the question, say you don't know.
"""

prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [144]:
 # Load and split PDFs
docs = load_and_split_pdfs_by_paragraphs(current_directory)
len(docs)

178

In [145]:
 # Initialize embedding model
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [146]:
 # Create Qdrant vector store
qdrant_vectorstore = Qdrant.from_documents(
    docs,
    embedding_model,
    location=":memory:",
    collection_name="extending_context_window_llama_3",
)

2024-09-21 10:23:13 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:13 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:14 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:16 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [147]:
# Create retriever
qdrant_retriever = qdrant_vectorstore.as_retriever()
  

In [148]:
# Create RAG prompt
rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

# Initialize OpenAI chat model
openai_chat_model = ChatOpenAI(model="gpt-4o-mini")

# Create the RAG chain
rag_chain = (
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    | rag_prompt
    | openai_chat_model
    | StrOutputParser()
)

In [149]:
rag_chain.invoke({"question" : "What are the pros and cons of AI being regulated?"})

2024-09-21 10:23:16 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:20 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'The context provided does not explicitly detail the pros and cons of AI regulation, but we can infer some potential advantages and disadvantages based on the principles and frameworks discussed. \n\n**Pros of AI Regulation:**\n1. **Protection of Rights and Civil Liberties:** Regulation can help ensure that AI systems do not infringe upon civil rights, civil liberties, and privacy, as highlighted in the Blueprint for an AI Bill of Rights.\n2. **Transparency and Accountability:** Regulations can promote transparency in AI systems, requiring them to be understandable and accountable, which can build public trust.\n3. **Safety and Security:** Regulatory frameworks can enforce safety standards for AI systems, similar to how the National Highway Traffic Safety Administration ensures vehicle safety, potentially leading to safer AI applications.\n4. **Guidance for Ethical Use:** Established ethical principles and frameworks, as seen with various U.S. government agencies, can guide the respons

## Synthetic Data Generation

In [150]:
# from langchain_community.document_loaders import PyMuPDFLoader

# documents = PyMuPDFLoader(file_path="https://s2.q4cdn.com/470004039/files/doc_earnings/2024/q3/filing/_10-Q-Q3-2024-As-Filed.pdf").load()

In [151]:
import os
from langchain_community.document_loaders import PyMuPDFLoader

# Directory where the PDF files are stored
directory_path = os.getcwd()

# Loop over all files in the directory and process PDF files
documents = []
for filename in os.listdir(directory_path):
    if filename.endswith(".pdf"):  # Check if the file is a PDF
        file_path = os.path.join(directory_path, filename)
        # Load the PDF using PyMuPDFLoader
        loader = PyMuPDFLoader(file_path=file_path)
        documents.extend(loader.load())  # Load and append documents to the list

# Now `documents` contains the content from all PDF files in the directory
print(f"Loaded {len(documents)} documents from the directory.")


Loaded 137 documents from the directory.


In [152]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

In [153]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

In [154]:
testset = generator.generate_with_langchain_docs(documents, 20, distributions, with_debugging_logs=True)


embedding nodes:   0%|          | 0/284 [00:00<?, ?it/s]

2024-09-21 10:23:23 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:23 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:23 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:23 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:23 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:23 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:23 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:23 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:23 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:23 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:24 - HTTP Request: POST

[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Automated systems', 'Equitable outcomes', 'Timely consideration', 'Effective organizational structure', 'Training and assessment']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Automated systems', 'Equitable outcomes', 'Timely consideration', 'Effective organizational structure', 'Training and assessment']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Automated systems', 'Equitable outcomes', 'Timely consideration', 'Effective organizational structure', 'Training and assessment']}


2024-09-21 10:23:24 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:23:24 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['AI system security', 'Resilience evaluation', 'Security measures', 'Content provenance', 'User surveys']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['AI system security', 'Resilience evaluation', 'Security measures', 'Content provenance', 'User surveys']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['AI system security', 'Resilience evaluation', 'Security measures', 'Content provenance', 'User surveys']}


...

[ragas.testset.evolutions.DEBUG] answer generated: {'answer': "Interpretability and explainability methods can improve GAI system decision evaluation by allowing stakeholders to understand how the system reaches its conclusions and verify that these decisions align with the intended purpose. This transparency helps identify and address any biases or errors in the system's decision-making process, ultimately enhancing trust and accountability.", 'verdict': 1}
[ragas.testset.evolutions.DEBUG] answer generated: {'answer': "Interpretability and explainability methods can improve GAI system decision evaluation by allowing stakeholders to understand how the system reaches its conclusions and verify that these decisions align with the intended purpose. This transparency helps identify and address any biases or errors in the system's decision-making process, ultimately enhancing trust and accountability.", 'verdict': 1}
[ragas.testset.evolutions.DEBUG] answer generated: {'answer': "Interpretab

2024-09-21 10:23:57 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:23:57 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ragas.testset.evolutions.DEBUG] answer generated: {'answer': "The criteria used to determine the relevance and quality of data in automated systems should be based on research-backed demonstration of the causal influence of the data to the specific use case or justified more generally based on a reasonable expectation of usefulness in the domain and/or for the system design or ongoing development. Relevance should not solely rely on historical connection to the outcome. High quality and tailored data should be representative of the task at hand, with errors measured and limited. Data used for predictions should have particular attention to quality and validity of the predicted outcome or label to ensure the automated system's goal is appropriately identified and measured. Justification should be documented for each data attribute and source to explain its appropriateness and compliance with applicable laws. For high-dimensional or derived attributes, overall descriptions of the attrib

2024-09-21 10:23:59 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[ragas.testset.evolutions.DEBUG] answer generated: {'answer': 'Stakeholders can ensure user understanding in automated systems by providing generally accessible plain language documentation that includes clear descriptions of the overall system functioning, the role of automation, the responsible entity, and explanations of outcomes. These notices should be kept up-to-date and people impacted by the system should be notified of significant changes. The documentation should be brief, clear, timely, and accessible, ensuring that users can easily find and understand the information. Additionally, explanations should be tailored to the specific purpose and audience, allowing users to build necessary understanding and achieve the stated purpose.', 'verdict': 1}
[ragas.testset.evolutions.DEBUG] answer generated: {'answer': 'Stakeholders can ensure user understanding in automated systems by providing generally accessible plain language documentation that includes clear descriptions of the ove

## Generated Data for Evaluation

In [155]:
testset.to_pandas()


,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What criteria should be used to determine the ...,[ \n \n \n \n \n \n \n \n \n \n \n \nSAFE AND ...,The criteria used to determine the relevance a...,simple,[{'source': '/Users/danielgutierrez/Midterm/Bl...,True
1,What stakeholders were involved in providing i...,[APPENDIX\n• OSTP conducted meetings with a va...,Stakeholders involved in providing ideas relat...,simple,[{'source': '/Users/danielgutierrez/Midterm/Bl...,True
2,What funding opportunities are available for C...,[ \nENDNOTES\n23. National Science Foundation....,Funding opportunities for Cyber-Physical Syste...,simple,[{'source': '/Users/danielgutierrez/Midterm/Bl...,True
3,What are the nine principles that federal agen...,[ \n \n \n \n \n \n \n \n \n \n \n \n \n \nSAF...,The nine principles that federal agencies must...,simple,[{'source': '/Users/danielgutierrez/Midterm/Bl...,True
4,How can structured feedback mechanisms be used...,[ \n29 \nMS-1.1-006 \nImplement continuous mon...,Structured feedback mechanisms can be used to ...,simple,[{'source': '/Users/danielgutierrez/Midterm/NI...,True
5,How can organizations enhance content provenan...,[ \n52 \n• \nMonitoring system capabilities an...,Organizations can enhance content provenance t...,simple,[{'source': '/Users/danielgutierrez/Midterm/NI...,True
6,How do healthcare navigators assist individual...,[ \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n...,Healthcare navigators assist individuals in fi...,simple,[{'source': '/Users/danielgutierrez/Midterm/Bl...,True
7,How can organizations verify information shari...,[ \n20 \nGV-4.3-003 \nVerify information shari...,Organizations can verify information sharing a...,simple,[{'source': '/Users/danielgutierrez/Midterm/NI...,True
8,What processes should be implemented to respon...,[ \n26 \nMAP 4.1: Approaches for mapping AI te...,Implement processes for responding to potentia...,simple,[{'source': '/Users/danielgutierrez/Midterm/NI...,True
9,How can bias in artificial intelligence be ide...,[ \n57 \nNational Institute of Standards and T...,Towards a Standard for Identifying and Managin...,simple,[{'source': '/Users/danielgutierrez/Midterm/NI...,True


In [156]:
testset_df = testset.to_pandas()

In [157]:
# from langsmith import Client

# client = Client()

# dataset_name = "AI Regulations Paragraph Chunks v3"

# dataset = client.create_dataset(
#     dataset_name=dataset_name,
#     description="Questions about AI regulations"
# )

In [158]:
# for test in testset.to_pandas().iterrows():
#   client.create_example(
#       inputs={
#           "question": test[1]["question"]
#       },
#       outputs={
#           "answer": test[1]["ground_truth"]
#       },
#       metadata={
#           "context": test[0]
#       },
#       dataset_id=dataset.id
#   )

## LangSmith Evaluation 

In [159]:
eval_llm = ChatOpenAI(model="gpt-4o")

In [160]:
# from langsmith.evaluation import LangChainStringEvaluator, evaluate

# qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

# labeled_helpfulness_evaluator = LangChainStringEvaluator(
#     "labeled_criteria",
#     config={
#         "criteria": {
#             "helpfulness": (
#                 "Is this submission helpful to the user,"
#                 " taking into account the correct reference answer?"
#             )
#         }
#     },
#     prepare_data=lambda run, example: {
#         "prediction": run.outputs["output"],
#         "reference": example.outputs["answer"],
#         "input": example.inputs["question"],
#     }
# )

# dope_or_nope_evaluator = LangChainStringEvaluator(
#     "criteria",
#     config={
#         "criteria": {
#             "dopeness": "Is this submission dope, lit, or cool?",
#         }
#     }
# )

In [161]:
# import time
# from openai import RateLimitError

# def evaluate_with_retry(rag_chain, dataset_name, retry_limit=5):
#     retry_count = 0
#     while retry_count < retry_limit:
#         try:
#             # Call the evaluate function
#             evaluate(
#                 rag_chain.invoke,
#                 data=dataset_name,
#                 evaluators=[
#                     qa_evaluator,
#                     labeled_helpfulness_evaluator,
#                     dope_or_nope_evaluator
#                 ],
#                 metadata={"revision_id": "default_chain"},
#             )
#             break  # If successful, break out of the loop
#         except RateLimitError as e:
#             # Parse the error to get the required wait time
#             error_message = str(e)
#             if "Rate limit reached" in error_message:
#                 # Extract the wait time from the error message (for example: 1.254s)
#                 wait_time = 1.254  # Default or extract from message if it's dynamic

#                 print(f"Rate limit exceeded. Waiting for {wait_time} seconds before retrying...")
#                 time.sleep(wait_time)
#                 retry_count += 1
#             else:
#                 raise  # If it's not a rate limit error, raise it

#     if retry_count == retry_limit:
#         raise Exception("Retry limit reached. Evaluation failed due to repeated rate limiting.")

# # Call the retry wrapper with the rag_chain and dataset name
# evaluate_with_retry(rag_chain, dataset_name)

## Evaluation og RAG Pipeline with RAGAS

In [162]:
# Take same SDG set from the baseline chunking
testset_df = pd.read_csv('testset_sdg.csv')

In [163]:
test_questions = testset_df["question"].values.tolist()
test_groundtruths = testset_df["ground_truth"].values.tolist()

In [164]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

2024-09-21 10:24:00 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:24:01 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:24:01 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:24:05 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:24:05 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:24:08 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:24:08 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:24:11 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:24:11 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:24:13 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21

In [165]:
#answers

In [166]:
#contexts

In [167]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [168]:
response_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 20
})

In [169]:
response_dataset[0]

{'question': 'How do healthcare navigators assist individuals in finding and obtaining healthcare coverage?',
 'answer': 'Healthcare navigators assist individuals by helping them navigate online signup forms to choose and obtain healthcare coverage. They are trained individuals or organizations that aid consumers, small businesses, and their employees in exploring health coverage options through the Marketplace, which is a government website. This includes assisting with the completion of eligibility and enrollment forms. For the 2022 plan year, the Biden-Harris Administration increased funding to train and certify more than 1,500 navigators to help uninsured consumers find affordable and comprehensive health coverage.',
 'contexts': ["HUMAN ALTERNATIVES, \nCONSIDERATION, AND \nFALLBACK \nHOW THESE PRINCIPLES CAN MOVE INTO PRACTICE\nReal-life examples of how these principles can become reality, through laws, policies, and practical \ntechnical and sociotechnical approaches to protectin

In [170]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [171]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

2024-09-21 10:25:09 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:25:09 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:25:10 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:25:10 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:25:10 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:25:10 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:25:10 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-21 10:25:10 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:25:11 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-09-21 10:25:11 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024

In [172]:
## 

In [173]:
results

{'faithfulness': 0.8881, 'answer_relevancy': 0.9631, 'context_recall': 0.8875, 'context_precision': 0.8736, 'answer_correctness': 0.7321}

In [174]:
results_df = results.to_pandas()
results_df.head()

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How do healthcare navigators assist individual...,"[HUMAN ALTERNATIVES, \nCONSIDERATION, AND \nFA...",Healthcare navigators assist individuals by he...,Healthcare navigators assist individuals in fi...,1.000000,0.943345,1.0,1.000000,0.834522
1,How are AI system performance or assurance cri...,[30 \nMEASURE 2.2: Evaluations involving human...,AI system performance or assurance criteria ar...,AI system performance or assurance criteria ar...,1.000000,0.979489,1.0,1.000000,0.649236
2,How can organizations enhance content provenan...,[52 \n• \nMonitoring system capabilities and l...,Organizations can enhance content provenance t...,Organizations can enhance content provenance t...,1.000000,0.972429,1.0,1.000000,0.513757
3,How can concerns of model collapse be mitigate...,[37 \nMS-2.11-005 \nAssess the proportion of s...,Concerns of model collapse can be mitigated by...,Assessing the proportion of synthetic to non-s...,0.750000,0.916980,0.5,0.805556,0.990257
4,What is the role of the NIST Generative AI Pub...,[NIST Trustworthy and Responsible AI \nNIST A...,The NIST Generative AI Public Working Group (G...,The answer to given question is not present in...,0.555556,0.970761,1.0,0.000000,0.926272


In [175]:
results_df.to_csv('results_alternative_chunking.csv', index=False)
